# CNN for Cifar-10 Dataset

## Requirements

For this project we are used:
* Python 3.6
* *Keras* from Tensorflow 1.8.0
* *GPUs parallel calculation manager* nVidia CUDA 9.0
* *GPU-accelerated library* nVidia cuDNN 7.1

## Introduction

In this first notebook we are going to explain how we have set up our work.  
Essentially the process is divided into several parts:
* Set up environment with the Cifar-10 Dataset
* Define a convolutional neural network
* Define a quantization method
* Train the convolutional neural network
* Provide information about CNN's performance and accuracy

## Cifar-10 Dataset
Cifar-10 Dataset is taken from the official website www.cs.toronto.edu.

Dataset is stored in the data directory: cnn/data. From Cifar-10 dataset we are going to take x_train, t_train, x_test and t_test.
The training dataset set is used for training the CNN, the testing dataset is used for evaluate the performance and the accuracy of the network.

In [1]:
"""This module provides utilities to download, init and load cifar10 dataset."""

from pathlib import Path

import h5py
import numpy as np
import tensorflow as tf
from tensorflow import keras

DATA_URL = 'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz'
DATA_DIR = 'cnn/data'
DATASET_FILE = 'cifar10.h5'


def load_dataset_as_tensors():
    """Load dataset from h5 (not lazily)."""

    h5_filename = Path(DATA_DIR) / DATASET_FILE
    with h5py.File(h5_filename.absolute(), 'r') as ds:
        return (ds['data/train'][()], ds['label/train'][()],
                ds['data/test'][()], ds['label/test'][()])


def preprocess_dataset(x_train, t_train, x_test, t_test, NCHW=False):
    """Preprocess dataset: label -> one hot encodeing, transpose if not in NCHW order."""

    t_train = keras.utils.to_categorical(t_train).astype(np.uint8)
    t_test = keras.utils.to_categorical(t_test).astype(np.uint8)

    if not NCHW:
        return (x_train.transpose(0, 2, 3, 1), t_train,
                x_test.transpose(0, 2, 3, 1), t_test)
    else:
        return (x_train, t_train,
                x_test, t_test)

/home/daibak/.virtualenvs/aca/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
x_train, t_train, x_test, t_test = load_dataset_as_tensors()

In [3]:
x_train.shape, t_train.shape, x_test.shape, t_test.shape

((50000, 3, 32, 32), (50000,), (10000, 3, 32, 32), (10000,))

Uniform data and visualize shape

In [4]:
x_train, t_train, x_test, t_test = preprocess_dataset(x_train, t_train, x_test, t_test)

In [5]:
x_train.shape, t_train.shape, x_test.shape, t_test.shape

((50000, 32, 32, 3), (50000, 10), (10000, 32, 32, 3), (10000, 10))

## CNN Model and Training

This CNN is called *dense_cnn*. Here we will explain how it is composed.

The CNN is composed by several layers. In the first part there are 2 **convolutional** layers and 2 **pooling** layers (they are alternated), then there are a *flatten* layer followed by a **relu** layer, a *dropout* layer and finally a **softmax** layer.

The network uses a stochastic gradient descent optimizer and a categorical crossentropy loss.  
To judge the performance of our model we are used a MSE metric.

In [ ]:
N_TRAIN_SAMPLES = x_train.shape[0]
N_TEST_SAMPLES = x_test.shape[0]
BATCH_SIZE = 64

model = keras.Sequential(
    name='dense_cnn',
    layers=[
        keras.layers.Conv2D(
            32,
            5,
            input_shape=(32, 32, 3),
            padding='same',
            activation=tf.nn.relu),
        keras.layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
        keras.layers.Conv2D(64, 5, padding='same', activation=tf.nn.relu),
        keras.layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
        keras.layers.Flatten(),
        keras.layers.Dense(1024, activation=tf.nn.relu),
        keras.layers.Dropout(0.4),
        keras.layers.Dense(10, activation=tf.nn.softmax)
    ])

model.compile(
    loss='categorical_crossentropy',
    optimizer='sgd',
    metrics=['accuracy', keras.metrics.mean_squared_error])

This network is trained for 50 epochs.

In [ ]:
hist = model.fit(
    x=x_train,
    y=t_train,
    batch_size=BATCH_SIZE,
    epochs=50,
    validation_split=0.2,
    shuffle=False,
    initial_epoch=0)

In [22]:
hist.history